In [3]:
import pandas as pd
import yfinance as yf
from collections.abc import Collection
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import numpy as np
from datetime import datetime, date, timedelta

In [ ]:
# TODO
# 1) this probably should pull stored values esp for back testing
# 2) maybe this should record the earliest date both assets have a price
def get_adj_close_history_df(tickers: list[str], start_date: str, end_date: str) -> pd.DataFrame:
    price_df = yf.download(
        tickers,
        start=start_date,
        end=end_date,
        progress=False,
        auto_adjust=True
    )['Close']
    return price_df.dropna()

# NOTE: This methodology came from Caldeira & Caldeira 2013. Paper is in references folder
def is_price_series_integrated_of_order_one(
        price_series: pd.Series,
        pvalue_threshold: float = 0.05
) -> bool:
    p_level = adfuller(price_series)[1]
    # Test if series is stationary
    if p_level < pvalue_threshold:
        return False

    # Test if series is stationary after applying one difference
    p_diff = adfuller(price_series.diff().dropna())[1]
    return True if p_diff < pvalue_threshold else False


def is_pair_engle_granger_cointegrated(
        ticker1_price_series: pd.Series,
        ticker2_price_series: pd.Series,
        pvalue_threshold: float = 0.05
) -> bool:
    _, pvalue, _ = coint(ticker1_price_series, ticker2_price_series)
    return True if pvalue < pvalue_threshold else False

# TODO underwrite det_order and k_ar_diff parameter values
# current parameter values are from chatgpt
# also corroborated here https://blog.quantinsti.com/johansen-test-cointegration-building-stationary-portfolio/
def is_pair_johansen_cointegrated(
        pair_price_history_df: pd.DataFrame
) -> bool:
    johansen_cointegration_result = coint_johansen(
        pair_price_history_df.values,
        det_order=0,
        k_ar_diff=1
    )
    index_of_95_pct_confidence_level_critical_value = 1
    index_of_trace_stat = 0
    trace_stat = johansen_cointegration_result.lr1[index_of_trace_stat]
    critical_value = johansen_cointegration_result.cvt[
        0,
        index_of_95_pct_confidence_level_critical_value
    ]
    return trace_stat > critical_value
    
def is_pair_tradable(
        ticker_pair: tuple[str],
        start_date: str,
        end_date: str
) -> bool:
    try:
        pair_price_history_df = get_adj_close_history_df(ticker_pair, start_date, end_date)
        ticker1_price_series = pair_price_history_df.iloc[:,0]
        ticker2_price_series = pair_price_history_df.iloc[:,1]
        if not is_price_series_integrated_of_order_one(ticker1_price_series):
            print(f"Pair {ticker_pair} first ticker series is not I(1)")
            return False
        if not is_price_series_integrated_of_order_one(ticker2_price_series):
            print(f"Pair {ticker_pair} second ticker series is not I(1)")
            return False
        if not is_pair_engle_granger_cointegrated(ticker1_price_series, ticker2_price_series):
            print(f"Pair {ticker_pair} is not engle granger cointegrated")
            return False
        if not is_pair_johansen_cointegrated(pair_price_history_df):
            print(f"Pair {ticker_pair} is not johansen cointegrated")
            return False
        return True
    except Exception as e:
        print(f"Unable to evaluate pair {ticker_pair}")
        print(e)
        return False
    
# TODO should eventually create a generic abstraction that takes
# a universe (list of tickers) and returns a collection of tradable pairs.
# But that computation is time consuming and I already have a list of candidate pairs
# so I'm going to start there for now.



# coints_df = pd.read_csv('../data/potential_pairs.csv')
# coints_df = coints_df[coints_df['pvalue'] < 0.02]
# coints_df['stock1'] = coints_df['pair'].str.split('_', expand=True)[0]
# coints_df['stock2'] = coints_df['pair'].str.split('_', expand=True)[1]
# pairs = list(zip(coints_df['stock1'], coints_df['stock2']))




In [18]:
# VALID PAIRS
('AAGH', 'CLF')
('AAGH', 'CIEN')
('AAGH', 'AMRN')
('AAGH', 'AMPH')
('AAGH', 'ATEX')

('AAGH', 'ATEX')

In [26]:
import pandas as pd
import requests

def get_sp_500_ticker_pairs() -> list[tuple]:
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) "
                            "Chrome/115.0 Safari/537.36"}

    res = requests.get(url, headers=headers)
    res.raise_for_status()

    tables = pd.read_html(res.text)
    sp500_table = tables[0]  # first table on the page
    tickers = sp500_table['Symbol'].to_list()

    pairs = []
    for ticker_one in tickers:
        for ticker_two in tickers:
            if ticker_one == ticker_two:
                continue

            pairs.append((ticker_one, ticker_two))

    return pairs



In [ ]:

valid_pairs = []
seen_pairs = []
start_date = '2015-1-1'
end_date = '2020-1-1'


pairs = get_sp_500_ticker_pairs()

for pair in pairs:
    if sorted(pair) in seen_pairs:
        continue
    if is_pair_tradable(pair, start_date, end_date):
        print("VALID:", pair)
        valid_pairs.append(pair)
    
        pd.DataFrame({"Valid Pairs": [pair]}).to_csv(
                    '../data/valid_sp500_pairs.csv',
                    mode="a",      # append mode
                    header=False,  # don’t write header again
                    index=False
                )
        
    seen_pairs.append(sorted(pair))





/var/folders/9t/v6gpfcbs71bcpv1bsh004rrc0000gn/T/ipykernel_63239/274187300.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(res.text)


Pair ('MMM', 'AOS') is not engle granger cointegrated
Pair ('MMM', 'ABT') is not engle granger cointegrated
Pair ('MMM', 'ABBV') is not engle granger cointegrated
Pair ('MMM', 'ACN') is not engle granger cointegrated
Pair ('MMM', 'ADBE') is not engle granger cointegrated
Pair ('MMM', 'AMD') is not engle granger cointegrated
Pair ('MMM', 'AES') is not engle granger cointegrated
Pair ('MMM', 'AFL') is not engle granger cointegrated
Pair ('MMM', 'A') is not engle granger cointegrated
Pair ('MMM', 'APD') is not engle granger cointegrated



1 Failed download:
['ABNB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-1-1 -> 2020-1-1) (Yahoo error = "Data doesn\'t exist for startDate = 1420088400, endDate = 1577854800")')


Unable to evaluate pair ('MMM', 'ABNB')
zero-size array to reduction operation maximum which has no identity
Pair ('MMM', 'AKAM') is not engle granger cointegrated
Pair ('MMM', 'ALB') is not engle granger cointegrated
Pair ('MMM', 'ARE') is not engle granger cointegrated
Pair ('MMM', 'ALGN') is not engle granger cointegrated
Pair ('MMM', 'ALLE') is not engle granger cointegrated
Pair ('MMM', 'LNT') is not engle granger cointegrated
Pair ('MMM', 'ALL') is not engle granger cointegrated
Pair ('MMM', 'GOOGL') is not engle granger cointegrated
Pair ('MMM', 'GOOG') is not engle granger cointegrated
Pair ('MMM', 'MO') is not engle granger cointegrated
Pair ('MMM', 'AMZN') is not engle granger cointegrated
Pair ('MMM', 'AMCR') is not engle granger cointegrated
Pair ('MMM', 'AEE') is not engle granger cointegrated
Pair ('MMM', 'AEP') is not engle granger cointegrated
Pair ('MMM', 'AXP') is not engle granger cointegrated
Pair ('MMM', 'AIG') is not engle granger cointegrated
Pair ('MMM', 'AMT') 


1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


...

Unable to evaluate pair ('ADBE', 'SOLV')
zero-size array to reduction operation maximum which has no identity
Pair ('ADBE', 'SO') is not engle granger cointegrated
Pair ('ADBE', 'LUV') is not engle granger cointegrated
Pair ('ADBE', 'SWK') is not engle granger cointegrated
Pair ('ADBE', 'SBUX') is not engle granger cointegrated
Pair ('ADBE', 'STT') is not engle granger cointegrated
Pair ('ADBE', 'STLD') is not engle granger cointegrated
Pair ('ADBE', 'STE') is not engle granger cointegrated
Pair ('ADBE', 'SYK') is not engle granger cointegrated
Pair ('ADBE', 'SMCI') is not engle granger cointegrated
Pair ('ADBE', 'SYF') is not engle granger cointegrated
Pair ('ADBE', 'SNPS') is not engle granger cointegrated
Pair ('ADBE', 'SYY') is not engle granger cointegrated
Pair ('ADBE', 'TMUS') is not engle granger cointegrated
Pair ('ADBE', 'TROW') is not engle granger cointegrated
Pair ('ADBE', 'TTWO') is not engle granger cointegrated
Pair ('ADBE', 'TPR') is not engle granger cointegrated
Pair


1 Failed download:
['VLTO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-1-1 -> 2020-1-1) (Yahoo error = "Data doesn\'t exist for startDate = 1420088400, endDate = 1577854800")')


Unable to evaluate pair ('ADBE', 'VLTO')
zero-size array to reduction operation maximum which has no identity
Pair ('ADBE', 'VRSN') is not engle granger cointegrated
Pair ('ADBE', 'VRSK') is not engle granger cointegrated
Pair ('ADBE', 'VZ') is not engle granger cointegrated
Pair ('ADBE', 'VRTX') is not engle granger cointegrated
Pair ('ADBE', 'VTRS') is not engle granger cointegrated
Pair ('ADBE', 'VICI') is not engle granger cointegrated
Pair ('ADBE', 'V') is not engle granger cointegrated
Pair ('ADBE', 'VST') is not engle granger cointegrated
Pair ('ADBE', 'VMC') is not engle granger cointegrated
Pair ('ADBE', 'WRB') is not engle granger cointegrated
Pair ('ADBE', 'GWW') is not engle granger cointegrated
Pair ('ADBE', 'WAB') is not engle granger cointegrated
Pair ('ADBE', 'WMT') is not engle granger cointegrated
Pair ('ADBE', 'DIS') is not engle granger cointegrated
Pair ('ADBE', 'WBD') is not engle granger cointegrated
Pair ('ADBE', 'WM') is not engle granger cointegrated
Pair ('AD


1 Failed download:
['ABNB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-1-1 -> 2020-1-1) (Yahoo error = "Data doesn\'t exist for startDate = 1420088400, endDate = 1577854800")')


Unable to evaluate pair ('AMD', 'ABNB')
zero-size array to reduction operation maximum which has no identity
Pair ('AMD', 'AKAM') is not engle granger cointegrated
Pair ('AMD', 'ALB') is not engle granger cointegrated
Pair ('AMD', 'ARE') is not engle granger cointegrated
Pair ('AMD', 'ALGN') is not engle granger cointegrated
VALID: ('AMD', 'ALLE')
Pair ('AMD', 'LNT') is not engle granger cointegrated
Pair ('AMD', 'ALL') is not engle granger cointegrated
Pair ('AMD', 'GOOGL') is not engle granger cointegrated
Pair ('AMD', 'GOOG') is not engle granger cointegrated
Pair ('AMD', 'MO') is not engle granger cointegrated
Pair ('AMD', 'AMZN') is not engle granger cointegrated
Pair ('AMD', 'AMCR') is not engle granger cointegrated
Pair ('AMD', 'AEE') is not engle granger cointegrated
Pair ('AMD', 'AEP') is not engle granger cointegrated
Pair ('AMD', 'AXP') is not engle granger cointegrated
Pair ('AMD', 'AIG') is not engle granger cointegrated
Pair ('AMD', 'AMT') is not johansen cointegrated
Pai

In [25]:
len(pairs)

252506